In [ ]:
# Cell 1: 환경 설정, 라이브러리, 데이터 로드 및 헬퍼 함수 정의

import torch
import numpy as np
import pandas as pd
import yaml
import csv
import os
import pickle
from scipy.stats import spearmanr
from tqdm.notebook import tqdm
import argparse
# from model import SkipGram  # 🚨 실제 SkipGram 클래스를 import 해야 합니다.

# ======================================================
# 1. 경로 설정 (필수 수정)
# ======================================================

# 🛠️ 이 경로들을 실제 파일 위치에 맞게 수정해주세요!
CONFIG_PATH = "../configs/skipgram_example.yaml"   
VOCAB_PATH = "../runs/vocab.pkl"                 
NS_CHECKPOINT_PATH = "../weights/sgns_model.pt"    # NS 모델 체크포인트
HS_CHECKPOINT_PATH = "../weights/sghs_model.pt"    # HS 모델 체크포인트

WORDSIM_CSV = "../data/wordsim/WordSim353_combined.csv"
SIMLEX_TXT = "../data/simlex/SimLex-999.txt"
GOOGLE_ANALOGY = "../data/analogy/questions-words.txt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🧠 Using device: {device}")

# ======================================================
# 2. Config & Vocabulary 로드
# ======================================================

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

with open(VOCAB_PATH, "rb") as f:
    vocab_data = pickle.load(f)
vocab, word2idx, idx2word = vocab_data["vocab"], vocab_data["word2idx"], vocab_data["idx2word"]

vocab_size = len(vocab)
embedding_dim = config["embedding_dim"]
print(f"📚 Loaded vocab: {vocab_size} words. Dim: {embedding_dim}")

# ======================================================
# 3. Data Loaders
# ======================================================

def load_wordsim353(path):
    pairs = []
    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            w1, w2, sim = row[0].lower(), row[1].lower(), float(row[2])
            pairs.append((w1, w2, sim))
    return pairs

def load_simlex999(path):
    pairs = []
    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')
        for row in reader:
            w1, w2, sim = row['word1'].lower(), row['word2'].lower(), float(row['SimLex999'])
            pairs.append((w1, w2, sim))
    return pairs

def load_google_analogy(path):
    analogies = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith(":") or len(line) == 0:
                continue
            words = line.lower().split()
            if len(words) == 4:
                analogies.append(tuple(words))
    return analogies

ws_pairs = load_wordsim353(WORDSIM_CSV)
simlex_pairs = load_simlex999(SIMLEX_TXT)
analogy_pairs = load_google_analogy(GOOGLE_ANALOGY)
print("✅ Evaluation datasets loaded.")

# ======================================================
# 4. Helper & Evaluation Functions
# ======================================================

# 🚨 주의: embedding_matrix는 각 모델 평가 셀에서 전역 변수로 정의해야 합니다!
embedding_matrix = None 

def get_embedding(word):
    global embedding_matrix
    if word not in word2idx:
        return None
    return embedding_matrix[word2idx[word]]

def cosine_similarity(v1, v2):
    return torch.dot(v1, v2) / (torch.norm(v1) * torch.norm(v2))

@torch.no_grad()
def analogy(a, b, c, topk=1):
    global embedding_matrix
    e_a, e_b, e_c = get_embedding(a), get_embedding(b), get_embedding(c)
    if e_a is None or e_b is None or e_c is None:
        return []
        
    target_vec = e_b - e_a + e_c
    target_vec = target_vec.half() 
    
    sims = torch.nn.functional.cosine_similarity(embedding_matrix, target_vec.unsqueeze(0), dim=1)

    for w in [a, b, c]:
        if w in word2idx:
            sims[word2idx[w]] = -float("inf")

    topk_idx = torch.topk(sims, topk).indices
    return [(idx2word[i.item()], sims[i].item()) for i in topk_idx]

@torch.no_grad()
def evaluate_similarity(pairs, desc):
    preds, golds = [], []
    for w1, w2, sim in tqdm(pairs, desc=f"🔹 {desc}", ncols=100):
        e1, e2 = get_embedding(w1), get_embedding(w2)
        if e1 is None or e2 is None:
            continue
        preds.append(cosine_similarity(e1, e2).cpu().item()) 
        golds.append(sim)
    corr, _ = spearmanr(preds, golds)
    oov_ratio = 1 - (len(preds) / len(pairs))
    print(f"  [Info] Evaluated pairs: {len(preds)}/{len(pairs)} (OOV: {oov_ratio:.2%})")
    return corr

@torch.no_grad()
def evaluate_analogy(analogies, desc):
    total, correct = 0, 0
    for a, b, c, d_true in tqdm(analogies, desc=f"🔸 {desc}", ncols=100):
        preds = analogy(a, b, c, topk=1)
        if not preds:
            continue
        total += 1
        if preds[0][0] == d_true:
            correct += 1
    return correct / total if total > 0 else 0.0

In [ ]:
# Cell 2: Skip-Gram Negative Sampling (SGNS) 모델 평가

print("=============== 🚀 SGNS 모델 평가 시작 ===============")

# 1. 모델 로드
try:
    ns_model = SkipGram(vocab_size, embedding_dim).to(device)
    ns_checkpoint = torch.load(NS_CHECKPOINT_PATH, map_location=device)
    ns_model.load_state_dict(ns_checkpoint["model_state_dict"])
    ns_model.eval()
    print(f"✅ NS Checkpoint 로드 완료: {NS_CHECKPOINT_PATH}")
except NameError:
    print("❌ SkipGram 클래스를 찾을 수 없습니다. Cell 1의 import를 확인하세요.")
    ns_results = {}
except Exception as e:
    print(f"❌ NS 모델 로드 실패: {e}")
    ns_results = {}
else:
    # 2. 임베딩 캐싱 (V + U 평균)
    print("\n⚡ Caching SGNS embeddings (V + U) / 2...")
    with torch.no_grad():
        in_weights = ns_model.in_embeddings.weight.detach().to(device)
        
        # NS 모드: V와 U를 평균내는 표준 방식 사용
        if hasattr(ns_model, 'out_embeddings') and ns_model.out_embeddings.weight.size(0) == vocab_size:
            out_weights = ns_model.out_embeddings.weight.detach().to(device)
            temp_matrix = (in_weights + out_weights) / 2
            print("💡 In (V)와 Out (U) 임베딩 평균 사용.")
        else:
            temp_matrix = in_weights
            print("⚠️ Out 임베딩을 찾을 수 없어 In (V) 임베딩만 사용.")
            
        # 전역 변수 업데이트 및 float16 변환
        global embedding_matrix
        embedding_matrix = temp_matrix.half() 
        
    print(f"✅ Embedding matrix cached. Size: {embedding_matrix.shape}, Dtype: {embedding_matrix.dtype}")
    
    # 3. 평가 실행
    ns_results = {}
    ns_results["WordSim-353"] = evaluate_similarity(ws_pairs, "WS-353 (NS)")
    ns_results["SimLex-999"] = evaluate_similarity(simlex_pairs, "SimLex-999 (NS)")
    ns_results["Google Analogy"] = evaluate_analogy(analogy_pairs, "Analogy (NS)")

    # 4. 결과 출력
    print("\n\n====================== SGNS 최종 결과 ======================")
    for name, val in ns_results.items():
        metric = "Spearman" if "Sim" in name else "Accuracy"
        print(f"📊 {name:<15}: {val:.4f} ({metric})")
    print("==========================================================")

In [ ]:
# Cell 3: Skip-Gram Hierarchical Softmax (SGHS) 모델 평가

print("=============== 🚀 SGHS 모델 평가 시작 ===============")

# 1. 모델 로드
try:
    hs_model = SkipGram(vocab_size, embedding_dim).to(device)
    hs_checkpoint = torch.load(HS_CHECKPOINT_PATH, map_location=device)
    hs_model.load_state_dict(hs_checkpoint["model_state_dict"])
    hs_model.eval()
    print(f"✅ HS Checkpoint 로드 완료: {HS_CHECKPOINT_PATH}")
except NameError:
    print("❌ SkipGram 클래스를 찾을 수 없습니다. Cell 1의 import를 확인하세요.")
    hs_results = {}
except Exception as e:
    print(f"❌ HS 모델 로드 실패: {e}")
    hs_results = {}
else:
    # 2. 임베딩 캐싱 (V만 사용)
    print("\n⚡ Caching SGHS embeddings (V only)...")
    with torch.no_grad():
        # HS 모드: V (입력 임베딩)만 평가에 사용합니다. (U는 내부 노드)
        in_weights = hs_model.in_embeddings.weight.detach().to(device)
        temp_matrix = in_weights
        print("💡 In (V) 임베딩만 사용 (HS 표준 방식).")
            
        # 전역 변수 업데이트 및 float16 변환
        global embedding_matrix
        embedding_matrix = temp_matrix.half() 
        
    print(f"✅ Embedding matrix cached. Size: {embedding_matrix.shape}, Dtype: {embedding_matrix.dtype}")

    # 3. 평가 실행
    hs_results = {}
    hs_results["WordSim-353"] = evaluate_similarity(ws_pairs, "WS-353 (HS)")
    hs_results["SimLex-999"] = evaluate_similarity(simlex_pairs, "SimLex-999 (HS)")
    hs_results["Google Analogy"] = evaluate_analogy(analogy_pairs, "Analogy (HS)")
    
    # 4. 결과 출력
    print("\n\n====================== SGHS 최종 결과 ======================")
    for name, val in hs_results.items():
        metric = "Spearman" if "Sim" in name else "Accuracy"
        print(f"📊 {name:<15}: {val:.4f} ({metric})")
    print("==========================================================")

In [ ]:
# Cell 4: 최종 결과 비교

print("=============== 🏆 전체 결과 비교 ================")

# ns_results와 hs_results가 정의되어 있는지 확인
if 'ns_results' in locals() and 'hs_results' in locals() and ns_results and hs_results:
    results_df = pd.DataFrame({
        "SGNS": ns_results,
        "SGHS": hs_results
    }).T
    
    results_df_styled = results_df.style.format("{:.4f}")
    display(results_df_styled)
else:
    print("❌ 모든 모델의 평가 결과가 존재하지 않습니다. 이전 셀의 로드 오류를 확인하세요.")

In [ ]:
# Cell 1의 상단 경로 설정 부분 수정 예시

# NS 모델 체크포인트 경로
NS_W5_CHECKPOINT_PATH = "../weights/sgns_w5_model.pt"  
NS_W10_CHECKPOINT_PATH = "../weights/sgns_w10_model.pt"

# HS 모델 체크포인트 경로
HS_W5_CHECKPOINT_PATH = "../weights/sghs_w5_model.pt"
HS_W10_CHECKPOINT_PATH = "../weights/sghs_w10_model.pt"
# ... (나머지 코드 동일)

In [ ]:
# Cell 3: SGNS (Window 10) 모델 평가

print("=============== 🚀 SGNS (W=10) 모델 평가 시작 ===============")
# ... (NS_W10_CHECKPOINT_PATH 로드) ...

# 💡 SGNS 임베딩 캐싱 로직 (V + U 평균)
# ... (ns_model 로드 및 V, U 평균 계산 후 embedding_matrix 업데이트) ...

# 평가 실행
ns_w10_results = {}
ns_w10_results["WordSim-353"] = evaluate_similarity(ws_pairs, "WS-353 (NS W=10)")
# ... (SimLex-999, Analogy 평가) ...

# 결과 출력 (ns_w10_results)
# ...

In [ ]:
# Cell 4: SGHS (Window 5) 모델 평가

print("=============== 🚀 SGHS (W=5) 모델 평가 시작 ===============")
# ... (HS_W5_CHECKPOINT_PATH 로드) ...

# 💡 SGHS 임베딩 캐싱 로직 (V만 사용)
# ... (hs_model 로드 및 V만 사용 후 embedding_matrix 업데이트) ...

# 평가 실행
hs_w5_results = {}
hs_w5_results["WordSim-353"] = evaluate_similarity(ws_pairs, "WS-353 (HS W=5)")
# ... (SimLex-999, Analogy 평가) ...

# 결과 출력 (hs_w5_results)
# ...

In [ ]:
# Cell 5: SGHS (Window 10) 모델 평가

print("=============== 🚀 SGHS (W=10) 모델 평가 시작 ===============")
# ... (HS_W10_CHECKPOINT_PATH 로드) ...

# 💡 SGHS 임베딩 캐싱 로직 (V만 사용)
# ... (hs_model 로드 및 V만 사용 후 embedding_matrix 업데이트) ...

# 평가 실행
hs_w10_results = {}
hs_w10_results["WordSim-353"] = evaluate_similarity(ws_pairs, "WS-353 (HS W=10)")
# ... (SimLex-999, Analogy 평가) ...

# 결과 출력 (hs_w10_results)
# ...

In [ ]:
# Cell 6: 최종 결과 비교

print("=============== 🏆 전체 결과 비교 ================")
# 모든 결과가 존재하는지 확인 후 DataFrame 생성

results_df = pd.DataFrame({
    "SGNS (W=5)": ns_w5_results,
    "SGNS (W=10)": ns_w10_results,
    "SGHS (W=5)": hs_w5_results,
    "SGHS (W=10)": hs_w10_results,
}).T 

results_df_styled = results_df.style.format("{:.4f}")
display(results_df_styled)